Exercise 4

In [11]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import time
import numba
import pylab

# to print plots inline
%matplotlib inline

In [12]:
# Specify parameters
# firms
alpha_k = 0.29715
alpha_l = 0.65
delta = 0.154
psi = 1.08
z = 1
sigma_z = 0.213
mu = 0.000
rho = 0.7605
sizez = 9

# households
h = 6.616
beta = 0.96

r_bar = 1/beta - 1
betafirm = 1/(1 + r_bar)


In [13]:
# Set up K-grid
dens = 1

# put in bounds here for the capital stock space
kstar = ((((1 / betafirm - 1 + delta) * ((0.7 / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (z ** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2 * kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
kvec = K[::-1]
sizek = kvec.shape[0]

# Set up Z-grid
import ar1_approx as ar1
num_sigma = 4
step = (num_sigma * sigma_z) / (sizez / 2)
pi, zvec = ar1.rouwen(rho, mu, step, sizez)
zvec = np.exp(zvec)

In [14]:
# operating profits, op
@numba.jit
def op(z, k, w):
    op = ((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l))) * ((k ** alpha_k) ** (1 / (1 - alpha_l)))) * z ** (1/(1 - alpha_l))
    return op

# firm cash flow, e
#numba.jit
def flow(w):
    e = np.zeros((sizek, sizek, sizez))
    for i in range(sizek):
        for j in range(sizek):
            for k in range(sizez):
                e[i, j, k] = (op(zvec[k], kvec[i], w) - kvec[j] + ((1 - delta) * kvec[i]) - 
                              ((psi / 2) * ((kvec[j] - ((1 - delta) * kvec[i])) ** 2)/ kvec[i]))
    return e

In [15]:
# labor demand
def get_ld(w, k, z):
    ld = (alpha_l/w) ** (1/(1-alpha_l)) * z ** (1/ (1 - alpha_l)) * k ** (alpha_k/(1-alpha_l))
    return ld

In [16]:
def c(w, k1, k):
    c = (psi / 2) * ((k1 - (1 - delta) * k)/k) ** 2 * k
    return c

In [17]:
# Define VFI function, which 
def VFI(w):
    VFtol = 1e-6
    VFdist = 7.0
    VFmaxiter = 3000
    V = np.zeros((sizek, sizez))  # initial guess of value function
    Vmat = np.zeros((sizek, sizek, sizez))  # initialize Vmat matrix
    VFiter = 1
    
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = V
        for i in range(sizek):  # loop over k
            for j in range(sizek):  # loop over k'
                for k in range(sizez): # loop over z
                    Vmat[i, j, k] = (flow(w))[i, j, k] + betafirm * np.dot(V[j, :], pi[:, k])
    
        # iteration for graphing later
        V = Vmat.max(axis=1)  # apply max operator to Vmat (to get V(k))

        PF = np.argmax(Vmat, axis=1)  # find the index of the optimal k'
        VFdist = (np.absolute(V - TV)).max()  # check distance between value
        # function for this iteration and value function from past iteration
        VFiter += 1
    if VFiter < VFmaxiter:
        print('Value function converged after this many iterations:', VFiter)
        print('Time taken for convergence:' + str(end - start))
    else:
        print('Value function did not converge')
        
    optK = kvec[PF]
    optI = optK - (1 - delta) * optK
    return PF, optK, optI

In [18]:
# Stationary distribution function, note that here pf is sizez by sizek
#@numba.jit
def stationary(w):
    PF, K, I = VFI(w)
    PF = PF.T
    Gamma = np.ones((sizez, sizek)) * (1 / (sizek * sizez))
    SDtol = 1e-12
    SDdist = 7
    SDiter = 0
    SDmaxiter = 1000
    while SDdist > SDtol and SDmaxiter > SDiter:
        HGamma = np.zeros((sizez, sizek))
        for i in range(sizez):  # z
            for j in range(sizek):  # k
                for m in range(sizez):  # z'
                    HGamma[m, PF[i, j]] = \
                        HGamma[m, PF[i, j]] + pi[i, m] * Gamma[i, j]
        SDdist = (np.absolute(HGamma - Gamma)).max()
        Gamma = HGamma
        SDiter += 1

    if SDiter < SDmaxiter:
        print('Stationary distribution converged after this many iterations: ',
              SDiter)
    else:
        print('Stationary distribution did not converge')
        
    return Gamma.T
    

In [19]:
# Aggregate Statistics
@numba.jit
def aggregate(w, gamma):
    PF, K, I = VFI(w)
    ldmat = np.zeros_like(K)
    cmat = np.zeros_like(K)
    ymat = np.zeros_like(K)
    for i in range(sizek):
        for j in range(sizez):
            ldmat[i, j] = get_ld(w, K[i, j], zvec[j])
            cmat[i, j] = c(w, K[i, j], kvec[i], zvec[j])
            ymat[i, j] = zvec[j] * kvec[i] ** alpha_k * ldmat[i, j] ** alpha_l
    ag_ld = np.sum(ldmat * gamma)
    ag_I = np.sum(I * gamma)
    ag_psi = np.sum(cmat * gamma)
    ag_Y = np.sum(ymat * gamma)
    hh_Consump = ag_Y - ag_I - ag_psi
    return ag_ld, hh_Consump

In [20]:
def get_ls(c, w):
    ls = c * w/h
    return ls

In [ ]:
from scipy.optimize import fminbound
# Begin iteration
tol = 1e-6
dist = 7.0
maxiter = 3000
ite = 1
w_old = 0.7 #initial guess
epsilon = 0.8

def l_dist(w):
    gamma = stationary(w)
    Ld, c = aggregate(w, gamma)
    Ls = get_ls(c, w_old)
    dist = abs(Ld - Ls)
    return dist

while ite < maxiter and dist > tol:
    w_new = w_old
    dist = l_dist(w_old)
    w = fminbound(l_dist, w_old, 1.5)
    w_old = epsilon * w_old + (1 - epsilon) * w
    ite += 1
    
print(w_old)   